In [2]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [3]:
start_date = '2019-06-01'
end_date = '2019-06-30'

# 集計
クロスデイバス・リタゲ配信が同居しているキャンペーンを抽出して全体を集計する

In [6]:
%%slack_notify @yusuke_akada
query = (
        f"""
        with xd_tmp as (
        select
        'XD' as castegory
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,ads.tactics_id
        ,hie.tactics_name
        ,seg.segment_name      
        ,ad_type
        ,case when ad_type = 0 then 'RTG'
        when ad_type = 1 then 'ATA'
        when ad_type = 2 then 'カスタム'
        when ad_type = 3 then '類似'
        when ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join dm.segment_hierarchies seg on ads.target_id = seg.target_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}'
        and (segment_name like '%XD%'
        or tactics_name like '%XD%')
        and ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
        order by yymmdd
        )
        ,
        all_tmp as (
        select
        'ALL' as category
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,ads.tactics_id
        ,hie.tactics_name
        ,"" as segment_name
        ,hie.ad_type
        ,case when hie.ad_type = 0 then 'RTG'
        when hie.ad_type = 1 then 'ATA'
        when hie.ad_type = 2 then 'カスタム'
        when hie.ad_type = 3 then '類似'
        when hie.ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        inner join xd_tmp on xd_tmp.strategy_id = ads.strategy_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}'
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
        order by yymmdd
        )
        select *
        from xd_tmp
        
        union all
        
        select *
        from all_tmp
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_shukei1 = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [7]:
df_shukei1.to_csv("201906_クロスデバイス集計.tsv", sep='\t', index=False)